# Customising UC2 Red Agents

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

This notebook will go over some examples of how red agent behaviour can be varied by changing its configuration parameters.

First, let's load the standard Data Manipulation config file, and see what the red agent does.

*(For a full explanation of the Data Manipulation scenario, check out the data manipulation scenario notebook)*

In [ ]:
# Imports

from primaite.config.load import data_manipulation_config_path
from primaite.game.agent.interface import AgentHistoryItem
from primaite.session.environment import PrimaiteGymEnv
import yaml
from pprint import pprint

In [ ]:
def make_cfg_have_flat_obs(cfg):
    for agent in cfg['agents']:
        if agent['type'] == "ProxyAgent":
            agent['agent_settings']['flatten_obs'] = False

In [ ]:
with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)
    make_cfg_have_flat_obs(cfg)

env = PrimaiteGymEnv(env_config = cfg)
obs, info = env.reset()
print('env created successfully')

In [ ]:
def friendly_output_red_action(info):
    # parse the info dict form step output and write out what the red agent is doing
    red_info : AgentHistoryItem = info['agent_actions']['data_manipulation_attacker']
    red_action = red_info.action
    if red_action == 'DONOTHING':
        red_str = 'DO NOTHING'
    elif red_action == 'NODE_APPLICATION_EXECUTE':
        client = "client 1" if red_info.parameters['node_id'] == 0 else "client 2"
        red_str = f"ATTACK from {client}"
    return red_str

By default, the red agent can start on client 1 or client 2. It starts its attack on a random step between 20 and 30, and it repeats its attack every 15-25 steps.

It also has a 20% chance to fail to perform the port scan, and a 20% chance to fail launching the SQL attack. However it will continue where it left off after a failed step. I.e. if lucky, it can perform the port scan and SQL attack on the first try. If the port scan works, but the sql attack fails the first time it tries to attack, the next time it will not need to port scan again, it can go straight to trying to use SQL attack again.

In [ ]:
for step in range(35):
    step_num = env.game.step_counter
    obs, reward, terminated, truncated, info = env.step(0)
    red = friendly_output_red_action(info)
    print(f"step: {step_num:3}, Red action: {friendly_output_red_action(info)}, Blue reward:{reward:.2f}" )

Since the agent does nothing most of the time, let's only print the steps where it performs an attack.

In [ ]:
env.reset()
for step in range(100):
    step_num = env.game.step_counter
    obs, reward, terminated, truncated, info = env.step(0)
    red = friendly_output_red_action(info)
    if red.startswith("ATTACK"):
        print(f"step: {step_num:3}, Red action: {friendly_output_red_action(info)}, Blue reward:{reward:.2f}" )

## Red Configuration

There are two important parts of the YAML config for varying red agent behaviour.

### Red agent settings
Here is an annotated config for the red agent in the data manipulation scenario.

```yaml
  - ref: data_manipulation_attacker  # name of agent
    team: RED # not used, just for human reference
    type: RedDatabaseCorruptingAgent  # type of agent - this lets primaite know which agent class to use

    # Since the agent does not need to react to what is happening in the environment, the observation space is empty.
    observation_space:
      type: UC2RedObservation
      options:
        nodes: {}

    action_space:
    
      # The agent has access to the DataManipulationBoth on clients 1 and 2.
      options:
        nodes:
        - node_name: client_1 # The network should have a node called client_1
          applications:
            - application_name: DataManipulationBot # The node client_1 should have DataManipulationBot configured on it
        - node_name: client_2 # The network should have a node called client_2
          applications:
            - application_name: DataManipulationBot # The node client_2 should have DataManipulationBot configured on it

        # not important
        max_folders_per_node: 1
        max_files_per_folder: 1
        max_services_per_node: 1

    # red agent does not need a reward function
    reward_function:
      reward_components:
        - type: DUMMY

    # These actions are passed to the RedDatabaseCorruptingAgent init method, they dictate the schedule of attacks
    agent_settings:
      start_settings:
        start_step: 25  # first attack at step 25
        frequency: 20  # attacks will happen every 20 steps (on average)
        variance: 5  # the timing of attacks will vary by up to 5 steps earlier or later
```

### Malicious application settings
The red agent uses an application called `DataManipulationBot` which leverages a node's `DatabaseClient` to send a malicious SQL query to the database server. Here's an annotated example of how this is configured in the yaml *(with impertinent config items omitted)*:

```yaml
simulation:
  network:
    nodes:
    - ref: client_1
      hostname: client_1
      type: computer
      ip_address: 192.168.10.21
      subnet_mask: 255.255.255.0
      default_gateway: 192.168.10.1
      
      # 
      applications:
      - ref: data_manipulation_bot
        type: DataManipulationBot
        options:
          port_scan_p_of_success: 0.8 # Probability that port scan is successful
          data_manipulation_p_of_success: 0.8 # Probability that SQL attack is successful
          payload: "DELETE" # The SQL query which causes the attack (this has to be DELETE)
          server_ip: 192.168.1.14 # IP address of server hosting the database
      - ref: client_1_database_client
        type: DatabaseClient # Database client must be installed in order for DataManipulationBot to function
        options:
          db_server_ip: 192.168.1.14 # IP address of server hosting the database
```

## Editing red agent settings

### Removing randomness from attack timing

We can make the attacks happen at completely predictable intervals if we edit the red agent's settings to set variance to 0.

In [ ]:
change = yaml.safe_load("""
start_settings:
  start_step: 25
  frequency: 20
  variance: 0
""")

with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)
    for agent in cfg['agents']:
      if agent['ref'] == "data_manipulation_attacker":
        agent['agent_settings'] = change

env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for step in range(100):
    step_num = env.game.step_counter
    obs, reward, terminated, truncated, info = env.step(0)
    red = friendly_output_red_action(info)
    if red.startswith("ATTACK"):
        print(f"step: {step_num:3}, Red action: {friendly_output_red_action(info)}, Blue reward:{reward:.2f}" )

### Making the start node always the same

Normally, the agent randomly chooses between the nodes in its action space to send attacks from:

In [ ]:
# Open the config without changing anything
with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)

env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for ep in range(12):
    env.reset()
    for step in range(31):
        step_num = env.game.step_counter
        obs, reward, terminated, truncated, info = env.step(0)
        red = friendly_output_red_action(info)
        if red.startswith("ATTACK"):
            print(f"Episode: {ep:2}, step: {step_num:3}, Red action: {friendly_output_red_action(info)}" )

We can make the agent always start on a node of our choice letting that be the only node in the agent's action space.

In [ ]:
change = yaml.safe_load("""
# TODO:
""")
#TODO 2869 fix

with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)
    for agent in cfg['agents']:
      if agent['ref'] == "data_manipulation_attacker":
        agent.update(change)

env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for ep in range(12):
    env.reset()
    for step in range(31):
        step_num = env.game.step_counter
        obs, reward, terminated, truncated, info = env.step(0)
        red = friendly_output_red_action(info)
        if red.startswith("ATTACK"):
            print(f"Episode: {ep:2}, step: {step_num:3}, Red action: {friendly_output_red_action(info)}" )

### Make the attack less likely to succeed.

We can change the success probabilities within the data manipulation bot application. When the attack succeeds, the reward goes down.

Setting the probabilities to 1.0 means the attack always succeeds - the reward will always drop

Setting the probabilities to 0.0 means the attack always fails - the reward will never drop.

In [ ]:
# Make attack always succeed.
change = yaml.safe_load("""
      applications:
      - ref: data_manipulation_bot
        type: DataManipulationBot
        options:
          port_scan_p_of_success: 1.0
          data_manipulation_p_of_success: 1.0
          payload: "DELETE"
          server_ip: 192.168.1.14
      - ref: client_1_web_browser
        type: WebBrowser
        options:
          target_url: http://arcd.com/users/
      - ref: client_1_database_client
        type: DatabaseClient
        options:
          db_server_ip: 192.168.1.14
""")

with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)
    cfg['simulation']['network']
    for node in cfg['simulation']['network']['nodes']:
      if node['hostname'] in ['client_1', 'client_2']:
        node['applications'] = change['applications']

env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for ep in range(5):
    env.reset()
    for step in range(36):
        step_num = env.game.step_counter
        obs, reward, terminated, truncated, info = env.step(0)
        red = friendly_output_red_action(info)
        if step_num == 35:
            print(f"Episode: {ep:2}, step: {step_num:3}, Reward: {reward:.2f}" )

In [ ]:
# Make attack always fail.
change = yaml.safe_load("""
      applications:
      - ref: data_manipulation_bot
        type: DataManipulationBot
        options:
          port_scan_p_of_success: 0.0
          data_manipulation_p_of_success: 0.0
          payload: "DELETE"
          server_ip: 192.168.1.14
      - ref: client_1_web_browser
        type: WebBrowser
        options:
          target_url: http://arcd.com/users/
      - ref: client_1_database_client
        type: DatabaseClient
        options:
          db_server_ip: 192.168.1.14
""")

with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)
    cfg['simulation']['network']
    for node in cfg['simulation']['network']['nodes']:
      if node['hostname'] in ['client_1', 'client_2']:
        node['applications'] = change['applications']

env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for ep in range(5):
    env.reset()
    for step in range(36):
        step_num = env.game.step_counter
        obs, reward, terminated, truncated, info = env.step(0)
        red = friendly_output_red_action(info)
        if step_num == 35:
            print(f"Episode: {ep:2}, step: {step_num:3}, Reward: {reward:.2f}" )